In [ ]:
import os
import pandas as pd
import numpy as np
import h5py
import seaborn as sns

def plot_and_save_histogram(data, column, result_name):
    """
    Plots a histogram for a specified column in the given DataFrame and saves it as a JPG file.

    Parameters:
    - data: DataFrame containing the data to be plotted.
    - column: Name of the column for which the histogram is to be plotted.
    - result_name: Name used for saving the histogram plot file.
    """
    fig = sns.displot(data=data, x=column, kde=True)
    fig.savefig(f'./{result_name}_hist_{column}.jpg')

def filter_and_save_data(file_name, result_name, thresholds):
    """
    Filters data from an HDF5 file generated from Picasso based on specified thresholds.
    Saves the filtered data as a CSV file and generates histograms for selected columns.

    Parameters:
    - file_name: Path to the HDF5 file generated from Picasso.
    - result_name: Name used for saving output files and histograms.
    - thresholds: A dictionary containing column names as keys and corresponding (lower, upper) threshold tuples.
    """
    # Reading hdf5 files from picasso
    df_initial = pd.DataFrame(np.array(h5py.File(file_name)['locs']))
    df_filtered = df_initial[["frame", "x", "y", "photons", "sx", "sy", "lpx", "lpy", "group"]]

    for column, (lower, upper) in thresholds.items():
        df_filtered = df_filtered[(df_filtered[column] > lower) & (df_filtered[column] < upper)]
        plot_and_save_histogram(df_filtered, column, result_name)

    # Save output
    df_filtered.to_csv(f'{result_name}_filtered.csv', index=False)

def process_files(file_paths, result_names, thresholds):
    """
    Processes multiple HDF5 files generated from Picasso, filters data, and generates histograms.

    Parameters:
    - file_paths: List of paths to HDF5 files generated from Picasso.
    - result_names: List of result names used for saving output files and histograms.
    - thresholds: A dictionary containing column names as keys and corresponding (lower, upper) threshold tuples.
    """
    for file_path, result_name in zip(file_paths, result_names):
        filter_and_save_data(file_path, result_name, thresholds)

if __name__ == "__main__":
    file_paths = [os.path.join('.', f'image{i}.hdf5') for i in range(1, 2)]
    result_names = [f'image_{i}' for i in range(1, 2)]

    thresholds = {
        'sx': (1.2, 2.2),
        'sy': (1.2, 2.2),
        'lpx': (0.3, float('inf')),  # Assuming upper_lpx is positive infinity
        'lpy': (0.3, float('inf'))   # Assuming upper_lpy is positive infinity
    }

    process_files(file_paths, result_names, thresholds)
